In [1]:
from pyspark.sql import SparkSession
from random import randint
from pyspark.sql.types import IntegerType, StructField, StructType

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName('SparkTitanic.com').getOrCreate()

In [4]:
train_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/train.csv')
    #for production: '../input/titanic/train.csv'

In [5]:
del(train_df) #end of training

In [6]:
def prediction(row):
    if row.Sex == 'female':
        return 1
    elif row.Sex == 'male':
        if row.Age == None:
            return randint(0,1)
        if row.Age < 15:
            return 1
        else:
            return 0 

In [7]:
test_df = spark.read.option("inferSchema", True)\
    .option("header", True)\
    .csv('./input/titanic/test.csv')
    #for production: '../input/titanic/test.csv'

In [8]:
output_df = spark.createDataFrame(spark.sparkContext.parallelize([]), StructType([
    StructField('PassengerId', IntegerType(), True),
    StructField('Survived', IntegerType(), True)
]))

In [9]:
for row in test_df.collect():
    output_df = output_df.union(spark.sparkContext.parallelize([(row.PassengerId, prediction(row))]).toDF(['PassengerId', 'Survived']))

In [10]:
output_df.toPandas().to_csv('./submission.csv', index=False)